In [ ]:
import os
import enum
import json
import httpx
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import shutil
from dotenv import load_dotenv
from datetime import date, timedelta, datetime
from typing import List, Dict, Union, Any, Optional, Date

load_dotenv()
pd.set_option('future.no_silent_downcasting', True)

In [ ]:
class PassportMOEXAuth:
    _instance: 'PassportMOEXAuth' = None

    def __new__(cls, *args, **kwargs) -> 'PassportMOEXAuth':
        """
        Создает и возвращает единственный экземпляр класса (реализует паттерн Singleton).
        """
        if cls._instance is None:
            cls._instance = super(PassportMOEXAuth, cls).__new__(cls)
        return cls._instance

    def __init__(self, username: str, password: str) -> None:
        """
        Инициализирует объект аутентификации с заданными именем пользователя и паролем.
        
        :param username: Имя пользователя для аутентификации
        :param password: Пароль для аутентификации
        """
        if not hasattr(self, 'initialized'):
            self.auth_cert: str | None = None
            self.error_count: int = 0
            self.username: str = username
            self.password: str = password
            self.initialized: bool = True

    def authenticate(self) -> bool:
        """
        Аутентифицирует пользователя на сервере MOEX и сохраняет аутентификационный сертификат.

        :return: Возвращает True, если аутентификация прошла успешно, иначе False.
        """
        AUTH_URL: str = 'https://passport.moex.com/authenticate'
        try:
            response: httpx.Response = httpx.get(AUTH_URL, auth=(self.username, self.password), timeout=600)
            if response.status_code == 200:
                self.auth_cert = response.cookies.get('MicexPassportCert')
                self.error_count = 0
                return True
            else:
                return False
        except httpx.RequestError as exc:
            print(f"An error occurred while requesting {exc.request.url!r}.")
            return False

    def auth_request(self, url: str) -> dict:
        """
        Выполняет авторизованный запрос к заданному URL и возвращает ответ в формате JSON.
        
        :param url: URL для выполнения запроса
        :return: Ответ в формате JSON
        :raises Exception: Если количество ошибок аутентификации превышает 3.
        """
        if self.auth_cert is None:
            self.authenticate()

        headers: dict[str, str] = {'Cookie': f'MicexPassportCert={self.auth_cert}', 'Cache-Control': 'no-cache'}
        
        try:
            response: httpx.Response = httpx.get(url, headers=headers, timeout=600) 
            if response.status_code == 200:
                self.error_count = 0
                return response.json()
            else:
                if self.error_count < 3:
                    self.authenticate()
                    self.error_count += 1
                    return self.auth_request(url=url)
                else:
                    raise Exception('Error while authorize passport.moex.com')
        except httpx.RequestError as exc:
            self.error_count += 1
            if self.error_count < 3:
                return self.auth_request(url=url)
            else:
                print(f"An error occurred while requesting {exc.request.url!r}.")
                return {}


In [ ]:
# Создаем экземпляр класса для авторизации запросов к ISS MOEX
auth = PassportMOEXAuth(username=os.getenv('LOGIN'), password=os.getenv('PASSWORD'))

In [ ]:
# Генератор дат
def date_range(start_date: date, end_date: date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

In [ ]:
# Маркеты и эндпоинты, по которым проходит валидация
class Markets(enum.Enum):
    """
    Типы рынков.
    """
    SHARES = 'eq'
    CURRENCY = 'fx'
    FUTURES = 'fo'


class Endpoints(enum.Enum):
    """
    API эндпоинты.
    """
    TRADESTATS = 'tradestats'
    ORDERSTATS = 'orderstats'
    ORDERBOOKSTATS = 'obstats'

In [ ]:
# Персеты для билда url для запроса к ISS
iss_market_url = {
        "eq": ("stock", "shares", "tqbr"),
        "fx": ("currency", "selt", "cets"),
        "fo": ("futures", "forts", "rfud")
    }

In [ ]:
# Создаем папку для отчетов
if os.path.exists('report'):
    shutil.rmtree('report')

os.makedirs(os.path.join('report', '5min_report'))

In [ ]:
# Функция определения торгового дня для маркета - обращается к исторической маркетдате и смотрит наличие сделок
def is_trading_day_for_market(market: Markets, trading_date: date):
    url = f"https://iss.moex.com/iss/history/engines/{iss_market_url[market][0]}/markets/{iss_market_url[market][1]}/boards/{iss_market_url[market][2]}/securities.json?date={trading_date.strftime('%Y-%m-%d')}"
    data = auth.auth_request(url=url)
    df = pd.DataFrame(data['history']['data'], columns=data['history']['columns'])
    return False if df["NUMTRADES"].sum() == 0 else True

In [ ]:
# Функция для получения количества сделок в торговом дне для маркета
def get_history_numtrades_for_market(market: Markets, trading_date: date):
    info = dict()
    url = f"https://iss.moex.com/iss/history/engines/{iss_market_url[market][0]}/markets/{iss_market_url[market][1]}/boards/{iss_market_url[market][2]}/securities.json?date={trading_date.strftime('%Y-%m-%d')}"
    data = auth.auth_request(url=url)
    df = pd.DataFrame(data['history']['data'], columns=data['history']['columns'])
    for secid in df['secid'].unique():
        info[secid] = df[df['secid'] == secid]['NUMTRADES'].sum()
    return info

In [ ]:
def read_json_file(file_path):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
            return data
    except (FileNotFoundError, json.JSONDecodeError):
        return []

In [ ]:
with open("missed_days.json", 'w') as f:
    pass


with open("tickers_count.json", 'w') as f:
    pass

In [ ]:
def slice_ticker_count_by_day(
    df: pd.DataFrame, 
    report_df: pd.DataFrame, 
    market: Markets, 
    trading_date: date
) -> pd.DataFrame:
    """
    Разделяет датафрейм по рынкам для подсчета количества тикеров в день для определенного рынка.

    :param df: Исходный датафрейм с данными о тикерах.
    :param report_df: Датафрейм для отчета, в который будут записываться результаты.
    :param market: Объект, представляющий рынок, для которого нужно выполнить расчет.
    :param trading_date: Дата, для которой производится расчет.
    :return: Обновленный датафрейм отчета с добавленной информацией о тикерах.
    """
    
    if df is None:
        return pd.DataFrame()
    
    if df.empty:
        report_df['secid'] = pd.Series(dtype=str)
        report_df.set_index('secid', inplace=True)
    
    info: Dict[str, int] = get_history_numtrades_for_market(market, trading_date)
    info_df: pd.DataFrame = pd.DataFrame(list(info.items()), columns=['secid', 'numtrades'])

    report_df[trading_date] = False
    
    valid_info_df: pd.DataFrame = info_df[info_df['numtrades'] != 0]

    for _, row in valid_info_df.iterrows():
        secid: str = row['secid']
        if secid in df['secid'].values:
            report_df.at[secid, trading_date] = True
        else:
            report_df.at[secid, trading_date] = False

    for secid in valid_info_df['secid'].values:
        if secid not in report_df.index:
            report_df.loc[secid] = {trading_date: False}

    report_df.index = report_df.index.astype(str)
    report_df.columns = pd.to_datetime(report_df.columns)

    return report_df

In [ ]:
def count_5min_tickers(
    df: pd.DataFrame, 
    market: str, 
    endpoint: str, 
    date: Union[datetime, str]
) -> pd.DataFrame:
    """
    Подсчитывает количество уникальных тикеров для каждой 5-минутки для заданного датафрейма, 
    рынка, конечной точки и даты.

    :param df: Датафрейм с данными о торговле.
    :param market: Название рынка.
    :param endpoint: Конечная точка.
    :param date: Дата торгов.
    :return: Датафрейм с количеством тикеров для каждого 5-минутного интервала.
    """
    df_tickers_count_store: pd.DataFrame = pd.DataFrame()
    df['tradedatetime'] = pd.to_datetime(df['tradedate'] + ' ' + df['tradetime'])
    df = df.set_index('tradedatetime')

    start_time: pd.Timestamp = df.index.min()
    end_time: pd.Timestamp = df.index.max()
    all_intervals: pd.DatetimeIndex = pd.date_range(start=start_time, end=end_time, freq='5min')

    for time_interval in all_intervals:
        interval_str: str = time_interval.strftime('%H:%M')
        interval_data: pd.DataFrame = df.between_time(time_interval.time(), (time_interval + pd.Timedelta(minutes=5)).time())
        ticker_count: int = interval_data['secid'].nunique() if not interval_data.empty else 0
        df_tickers_count: pd.DataFrame = pd.DataFrame({
            'date': [date], 
            'market': [market], 
            'endpoint': [endpoint], 
            'interval': [interval_str], 
            'ticker_count': [ticker_count]
        })
        df_tickers_count_store = pd.concat([df_tickers_count_store, df_tickers_count])

    return df_tickers_count_store

In [ ]:
def plot_tickers_count_by_day(
    data: pd.DataFrame, 
    markets: List[str] = ['fx', 'fo', 'eq'], 
    filepath: Optional[str] = None
) -> None:
    """
    Строит график количества тикеров по дням и типам рынков.

    :param data: Датафрейм с данными о тикерах.
    :param markets: Список рынков для фильтрации данных.
    :param filepath: Путь для сохранения графика. Если None, график будет отображен.
    """
    data_filtered: pd.DataFrame = data[data['market'].isin(markets)]
    data_filtered['date'] = pd.to_datetime(data_filtered['date'])
    dates: List[pd.Timestamp] = sorted(data_filtered['date'].unique())
    market_data: dict = {market: {stats: [] for stats in data_filtered['endpoint'].unique()} for market in markets}
    
    grouped: pd.DataFrame = data_filtered.groupby(['date', 'market', 'endpoint']).sum().reset_index()
    for date in dates:
        for market in markets:
            for stats in data_filtered['endpoint'].unique():
                value: pd.Series = grouped[(grouped['date'] == date) & (grouped['market'] == market) & (grouped['endpoint'] == stats)]['count']
                if not value.empty:
                    market_data[market][stats].append(value.values[0])
                else:
                    market_data[market][stats].append(0)

    _, ax = plt.subplots(figsize=(15, 10))
    
    for market in markets:
        for stats in data_filtered['endpoint'].unique():
            if any(market_data[market][stats]):
                ax.plot(dates, market_data[market][stats], marker='o', label=f'{market.upper()} - {stats}')

    ax.set_title('Количество тикеров по дням и типам рынков')
    ax.set_xlabel('Дни')
    ax.set_ylabel('Количество тикеров')
    
    ax.set_xticks(dates)
    ax.set_xticklabels([date.strftime('%Y-%m-%d') for date in dates], rotation=90)
    
    ax.legend(loc='upper right', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
    ax.grid(True)

    table_data: List[List[Union[str, float]]] = []
    for market in markets:
        for stats in data_filtered['endpoint'].unique():
            if any(market_data[market][stats]):
                min_value: float = min(market_data[market][stats])
                max_value: float = max(market_data[market][stats])
                average_value: float = sum(market_data[market][stats]) / len(market_data[market][stats])
                median_value: float = statistics.median(market_data[market][stats])

                table_data.append([f'{market.upper()} - {stats}', min_value, max_value, average_value, median_value])

    table_columns: List[str] = ["Market - Stats", "Min", "Max", "Average", "Median"]
    table = plt.table(cellText=table_data, colLabels=table_columns, cellLoc='center', loc='bottom', bbox=[0, -0.5, 1, 0.3])

    table.auto_set_font_size(False)
    table.set_fontsize(10)
    table.scale(1, 1.5)

    plt.subplots_adjust(bottom=0.3, right=0.8)

    plt.tight_layout()
    
    if filepath is not None:
        plt.savefig(filepath)
        plt.close()
    else:
        plt.show()
        plt.close()

In [ ]:
def plot_5min_tickers(
    df: pd.DataFrame, 
    date: date, 
    market: str, 
    endpoint: str, 
    filepath: Optional[str] = None
) -> None:
    """
    Строит график количества тикеров по 5-минутным интервалам для заданной даты, рынка и конечной точки.

    :param df: Датафрейм с данными о тикерах.
    :param date: Дата для фильтрации данных.
    :param market: Название рынка для фильтрации данных.
    :param endpoint: Конечная точка для фильтрации данных.
    :param filepath: Путь для сохранения графика. Если None, график будет отображен.
    """
    filtered_df: pd.DataFrame = df[(df['date'] == date.strftime('%Y-%m-%d')) & (df['market'] == market) & (df['endpoint'] == endpoint)]
    
    if filtered_df.empty:
        print(f"Нет данных для {date}, {market}, {endpoint}")
        return
    
    plt.figure(figsize=(15, 5))
    plt.plot(filtered_df['interval'], filtered_df['ticker_count'], marker='o', linestyle='-', color='b')
    
    plt.title(f"Количество тикеров по 5-минуткам для {date}, {market}, {endpoint}")
    plt.xlabel("Интервал")
    plt.ylabel("Количество тикеров")
    plt.xticks(rotation=90)
    plt.grid(True)

    min_count: float = filtered_df['ticker_count'].min()
    max_count: float = filtered_df['ticker_count'].max()
    avg_count: float = filtered_df['ticker_count'].mean()
    med_count: float = filtered_df['ticker_count'].median()
    
    stats_data: list[list[float | str]] = [
        ['Min', min_count], 
        ['Max', max_count], 
        ['Average', avg_count], 
        ['Median', med_count]
    ]
    plt.table(
        cellText=stats_data, 
        colLabels=['Statistic', 'Value'], 
        cellLoc='center', 
        loc='bottom', 
        bbox=[0, -0.7, 1, 0.35]
    )
    
    plt.subplots_adjust(bottom=0.3)
    plt.tight_layout()

    if filepath is not None:
        plt.savefig(filepath)
        plt.close()
    else:
        plt.show()
        plt.close()


In [ ]:
def check_negative_values(series: pd.Series, metric: str) -> Dict[Any, str]:
    """
    Проверяет наличие отрицательных значений или значений None в серии и возвращает словарь с индексами таких значений и метрикой.

    :param series: Серия pandas для проверки.
    :param metric: Строка, представляющая метрику, которая будет добавлена в результат.
    :return: Словарь, где ключи - это индексы значений None или отрицательных значений, а значения - это метрика.
    """
    result: Dict[Any, str] = {}
    for index, value in series.items():
        if value is None or value < 0:
            result[index] = metric

    return result

In [ ]:
def check_range(series: pd.Series, metric: str) -> Dict[Any, str]:
    """
    Проверяет значения в серии на соответствие заданному диапазону [-1, 1]. 
    Возвращает словарь с индексами значений, выходящих за пределы диапазона, и метрикой.

    :param series: Серия pandas, содержащая значения для проверки.
    :param metric: Строка, представляющая метрику, которая будет добавлена в результат.
    :return: Словарь, где ключи - это индексы значений, выходящих за пределы диапазона [-1, 1], а значения - это метрика.
    """
    out_of_range_values: Dict[Any, str] = {}
    for index, value in series.items():
        if value is not None and (value < -1 or value > 1):
            out_of_range_values[index] = metric

    return out_of_range_values

In [ ]:
def validate_range_negative_data(
    df: pd.DataFrame,
    market: str,
    endpoint: str
) -> Union[bool, pd.DataFrame]:
    """
    Проверяет значения в датафрейме на соответствие диапазону и отрицательным значениям 
    в зависимости от типа рынка и конечной точки. Возвращает датафрейм с результатами проверки 
    или False, если нет данных для проверки.

    :param df: Датафрейм с данными для проверки.
    :param market: Тип рынка (например, 'SHARES', 'CURRENCY', 'FUTURES').
    :param endpoint: Конечная точка (например, 'TRADESTATS', 'ORDERSTATS', 'ORDERBOOKSTATS').
    :return: Датафрейм с результатами проверки или False, если нет данных для проверки.
    """
    result_list: List[Dict[str, Union[str, Dict[str, str]]]] = []

    grouped_dfs: pd.DataFrameGroupBy = df.groupby('secid')

    for secid, group_df in grouped_dfs:
        columns_to_validate: List[str] = []

        if market == Markets.SHARES.value and endpoint == Endpoints.TRADESTATS.value:
            columns_to_validate = [
                'pr_open', 'pr_high', 'pr_low', 'pr_close', 'pr_std', 'vol', 'val', 'trades', 'pr_vwap',
                'trades_b', 'trades_s', 'val_b', 'val_s', 'vol_b', 'vol_s', 'pr_vwap_b', 'pr_vwap_s', 'disb'
            ]
        elif market == Markets.SHARES.value and endpoint == Endpoints.ORDERSTATS.value:
            columns_to_validate = [
                'put_orders_b', 'put_orders_s', 'put_val_b', 'put_val_s', 'put_vol_b', 'put_vol_s',
                'put_vwap_b', 'put_vwap_s', 'put_vol', 'put_val', 'put_orders', 'cancel_orders_b',
                'cancel_orders_s', 'cancel_val_b', 'cancel_val_s', 'cancel_vol_b', 'cancel_vol_s',
                'cancel_vwap_b', 'cancel_vwap_s', 'cancel_vol', 'cancel_val', 'cancel_orders'
            ]
        elif market == Markets.SHARES.value and endpoint == Endpoints.ORDERBOOKSTATS.value:
            columns_to_validate = [
                'spread_bbo', 'spread_lv10', 'spread_1mio', 'levels_b', 'levels_s', 'vol_b',
                'vol_s', 'val_b', 'val_s', 'imbalance_vol_bbo', 'imbalance_val_bbo', 'imbalance_vol',
                'imbalance_val', 'vwap_b', 'vwap_s', 'vwap_b_1mio', 'vwap_s_1mio'
            ]
        elif market == Markets.CURRENCY.value and endpoint == Endpoints.TRADESTATS.value:
            columns_to_validate = [
                'pr_open', 'pr_close', 'pr_high', 'pr_low', 'pr_std', 'vol', 'val', 'trades',
                'pr_vwap', 'trades_b', 'trades_s', 'val_b', 'val_s', 'vol_b', 'vol_s', 'disb'
            ]
        elif market == Markets.CURRENCY.value and endpoint == Endpoints.ORDERSTATS.value:
            columns_to_validate = [
                'put_orders_b', 'put_orders_s', 'put_val_b', 'put_val_s', 'put_vol_b', 'put_vol_s',
                'put_vwap_b', 'put_vwap_s', 'cancel_orders_b', 'cancel_orders_s', 'cancel_val_b',
                'cancel_val_s', 'cancel_vol_b', 'cancel_vol_s', 'cancel_vwap_b', 'cancel_vwap_s'
            ]
        elif market == Markets.CURRENCY.value and endpoint == Endpoints.ORDERBOOKSTATS.value:
            columns_to_validate = [
                'spread_l1', 'spread_l2', 'spread_l3', 'spread_l5', 'spread_l10', 'levels_b',
                'levels_s', 'vol_b_l1', 'vol_b_l2', 'vol_b_l3', 'vol_b_l5', 'vol_b_l10', 'vol_s_l1',
                'vol_s_l2', 'vol_s_l3', 'vol_s_l5', 'vol_s_l10', 'vol_s_l20', 'vwap_b_l3', 'vwap_b_l5',
                'vwap_b_l10', 'vwap_s_l3', 'vwap_s_l5', 'vwap_s_l10'
            ]
        elif market == Markets.FUTURES.value and endpoint == Endpoints.TRADESTATS.value:
            columns_to_validate = [
                'pr_open', 'pr_high', 'pr_low', 'pr_close', 'pr_std', 'vol', 'val', 'trades', 'pr_vwap',
                'trades_b', 'trades_s', 'val_b', 'val_s', 'vol_b', 'vol_s', 'disb', 'pr_vwap_b', 'pr_vwap_s'
            ]
        elif market == Markets.FUTURES.value and endpoint == Endpoints.ORDERBOOKSTATS.value:
            columns_to_validate = [
                'mid_price', 'micro_price', 'spread_l1', 'spread_l2', 'spread_l3', 'spread_l5', 'spread_l10',
                'spread_l20', 'levels_b', 'levels_s', 'vol_b_l1', 'vol_b_l2', 'vol_b_l3', 'vol_b_l5', 'vol_b_l10',
                'vol_b_l20', 'vol_s_l1', 'vol_s_l2', 'vol_s_l3', 'vol_s_l5', 'vol_s_l10', 'vol_s_l20', 'vwap_b_l3',
                'vwap_b_l5', 'vwap_b_l10', 'vwap_b_l20', 'vwap_s_l3', 'vwap_s_l5', 'vwap_s_l10', 'vwap_s_l20'
            ]

        invalid_columns: List[Dict[str, str]] = []
                    
        for metric in columns_to_validate:
            if 'disb' == metric or 'imbalance' in metric:
                issue = check_range(group_df[metric], metric)
            else:
                issue = check_negative_values(group_df[metric], metric)

            if issue:
                issue = {group_df.at[index, 'tradetime']: value for index, value in issue.items()}
                if issue:
                    invalid_columns.append(issue)

        if invalid_columns:
            for invalid_column in invalid_columns:
                result_list.append({
                    'date': df['date'].iloc[0],
                    'market': market,
                    'endpoint': endpoint,
                    'secid': secid,
                    'invalid_column': invalid_column
                })

    negative_range_report: pd.DataFrame = pd.DataFrame(result_list)
    
    if 'invalid_column' not in negative_range_report.columns:
        return False
    
    negative_range_report = negative_range_report[negative_range_report['invalid_column'] != True]
    subset_columns: List[str] = ["date", "market", "endpoint", "secid"]
    negative_range_report = negative_range_report.drop_duplicates(subset=subset_columns)
    
    return negative_range_report

In [ ]:
def get_decimals():
    """
    Получает количество знаков после запятой для каждого тикера на различных рынках.

    Возвращает:
    ----------
    tuple
        Кортеж, содержащий три словаря: для акций, валют и фьючерсов.
    """
    eq_decimals = dict()
    fx_decimals = dict()
    fo_decimals = dict()

    def update_decimals(url, decimals_dict):
        data = auth.auth_request(url=url)
        
        columns = data['securities']['columns']
        data_rows = data['securities']['data']
        
        secid_index = columns.index('SECID')
        decimals_index = columns.index('DECIMALS')
        
        for row in data_rows:
            secid = row[secid_index]
            decimals = row[decimals_index]
            decimals_dict[secid] = decimals

    eq_url = 'https://iss.moex.com/iss/engines/stock/markets/shares/boards/tqbr/securities.json'
    fx_url = 'https://iss.moex.com/iss/engines/currency/markets/selt/boards/cets/securities.json'
    fo_url = 'https://iss.moex.com/iss/engines/futures/markets/forts/boards/rfud/securities.json'

    update_decimals(eq_url, eq_decimals)
    update_decimals(fx_url, fx_decimals)
    update_decimals(fo_url, fo_decimals)

    return eq_decimals, fx_decimals, fo_decimals

eq_decimals, fx_decimals, fo_decimals = get_decimals()
tickers_decimal = eq_decimals | fx_decimals | fo_decimals
tickers_decimal['SBER']

In [ ]:
def validate_rounding(series: pd.Series, decimal: int) -> bool:
    """
    Проверяет, правильно ли округлены значения в серии данных.

    Параметры:
    ----------
    series : pd.Series
        Серия данных для проверки округления.
    decimal : int
        Количество знаков после запятой для округления.

    Возвращает:
    ----------
    bool
        True, если все значения правильно округлены, в противном случае False.
    """
    for value in series:
        rounded_value = round(value, decimal)
        if rounded_value != value:
            return False
    return True


a = pd.Series([1.22])

validate_rounding(series=a, decimal=3)

In [ ]:
def validate_decimals(dataset: Dict[str, Dict[str, Dict[str, pd.DataFrame]]]) -> pd.DataFrame:
    """
    Проверяет, что значения в датафреймах округлены до корректного числа десятичных знаков в зависимости от типа рынка и конечной точки.

    :param dataset: Словарь, содержащий данные для проверки. Ключи - даты, рынки и конечные точки, значения - датафреймы с данными.
    :return: Датафрейм с результатами проверки. Каждый результат включает дату, рынок, конечную точку, идентификатор секьюрити и некорректные столбцы.
    """
    result_list: List[Dict[str, Union[str, int]]] = []

    for date, markets in dataset.items():
        for market, endpoints in markets.items():
            for endpoint, df in endpoints.items():
                print(f"{date} | {market} | {endpoint}")

                grouped_dfs = df.groupby('secid')

                for secid, group_df in grouped_dfs:
                    decimal_places = tickers_decimal.get(secid, 2)
                    group_df.fillna(0, inplace=True)

                    columns_to_validate = {}

                    if market == Markets.SHARES.value:
                        if endpoint == Endpoints.TRADESTATS.value:
                            columns_to_validate = {
                                "pr_open": decimal_places, "pr_high": decimal_places, "pr_low": decimal_places,
                                "pr_close": decimal_places, "pr_change": 4, "pr_vwap": decimal_places,
                                "pr_vwap_b": decimal_places, "pr_vwap_s": decimal_places, "pr_std": 4, "disb": 2
                            }
                        elif endpoint == Endpoints.ORDERSTATS.value:
                            columns_to_validate = {
                                "put_vwap_b": decimal_places, "put_vwap_s": decimal_places,
                                "cancel_vwap_b": decimal_places, "cancel_vwap_s": decimal_places,
                            }
                        elif endpoint == Endpoints.ORDERBOOKSTATS.value:
                            columns_to_validate = {
                                "mid_price": decimal_places, "micro_price": decimal_places,
                                "spread_l1": decimal_places, "spread_l2": decimal_places,
                                "spread_l3": decimal_places, "spread_l5": decimal_places,
                                "spread_l10": decimal_places, "vwap_b_l3": decimal_places,
                                "vwap_b_l5": decimal_places, "vwap_b_l10": decimal_places,
                                "vwap_s_l3": decimal_places, "vwap_s_l5": decimal_places,
                                "vwap_s_l10": decimal_places,
                            }

                    elif market == Markets.CURRENCY.value:
                        if endpoint == Endpoints.TRADESTATS.value:
                            columns_to_validate = {
                                "pr_open": decimal_places, "pr_high": decimal_places, "pr_low": decimal_places,
                                "pr_close": decimal_places, "pr_change": 4, "pr_vwap": decimal_places,
                                "pr_vwap_b": decimal_places, "pr_vwap_s": decimal_places, "pr_std": 4, "disb": 2
                            }
                        elif endpoint == Endpoints.ORDERSTATS.value:
                            columns_to_validate = {
                                "put_vwap_b": decimal_places, "put_vwap_s": decimal_places,
                                "cancel_vwap_b": decimal_places, "cancel_vwap_s": decimal_places,
                            }
                        elif endpoint == Endpoints.ORDERBOOKSTATS.value:
                            columns_to_validate = {
                                "vwap_b": decimal_places, "vwap_s": decimal_places,
                                "vwap_b_1mio": decimal_places, "vwap_s_1mio": decimal_places,
                            }

                    elif market == Markets.FUTURES.value:
                        if endpoint == Endpoints.TRADESTATS.value:
                            columns_to_validate = {
                                "pr_open": decimal_places, "pr_high": decimal_places, "pr_low": decimal_places,
                                "pr_close": decimal_places, "pr_change": 4, "pr_vwap": decimal_places,
                                "pr_vwap_b": decimal_places, "pr_vwap_s": decimal_places, "pr_std": 4, "disb": 2
                            }
                        elif endpoint == Endpoints.ORDERBOOKSTATS.value:
                            columns_to_validate = {
                                "mid_price": decimal_places, "micro_price": decimal_places,
                                "spread_l1": 1, "spread_l2": 1, "spread_l3": 1, "spread_l5": 1, "spread_l10": 1,
                                "spread_l20": 1, "vwap_b_l3": decimal_places, "vwap_b_l5": decimal_places,
                                "vwap_b_l10": decimal_places, "vwap_b_l20": decimal_places,
                                "vwap_s_l3": decimal_places, "vwap_s_l5": decimal_places,
                                "vwap_s_l10": decimal_places, "vwap_s_l20": decimal_places,
                            }

                    invalid_columns = []

                    for column, decimal in columns_to_validate.items():
                        if column in group_df.columns and not validate_rounding(group_df[column], decimal):
                            invalid_columns.append(column)

                    if invalid_columns:
                        for column in invalid_columns:
                            result_list.append({
                                'date': date,
                                'market': market,
                                'endpoint': endpoint,
                                'secid': secid,
                                'invalid_column': column
                            })

    return pd.DataFrame(result_list)

In [ ]:
def get_dataset(start: date, end: date):
    eq_slice_reports_df = None
    fx_slice_reports_df = None
    fo_slice_reports_df = None

    sliced_reports = {
        "eq": [eq_slice_reports_df, f"sliced_eq_reports_{start}_{end}.csv"],
        "fx": [fx_slice_reports_df, f"sliced_fx_reports_{start}_{end}.csv"],
        "fo": [fo_slice_reports_df, f"sliced_fo_reports_{start}_{end}.csv"],
    }

    # Проходим по каждому дню маркету и эндпоинту
    for trading_date in date_range(start_date=start, end_date=end):
        formatted_date = trading_date.strftime('%Y-%m-%d')

        for market in Markets:
            for endpoint in Endpoints:
                # Пропускаем FO orderstats
                if market == Markets.FUTURES and endpoint == Endpoints.ORDERSTATS:
                    continue
                
                trading_day_status = is_trading_day_for_market(market=market.value, trading_date=trading_date)

                # Если не торговый день или не было сделок - то скипаем
                if not trading_day_status:
                    continue

                url = f'https://iss.moex.com/iss/datashop/algopack/{market.value}/{endpoint.value}.json?date={formatted_date}'
                
                all_data = []
                page_num = 0
                
                # Получаем данные за день
                while True:
                    page_url = f'{url}&start={page_num * 1000}'
                    data = auth.auth_request(url=page_url)
                    columns = data['data']['columns']
                    page_data = data['data']['data']
                    
                    if not page_data or page_data is None:
                        break
                    
                    all_data.extend(page_data)
                    page_num += 1
                    print(f"{market.value} | {endpoint.value} | {formatted_date}: page num: ", page_num, end='\n')

                df = pd.DataFrame(all_data, columns=columns)
                
                # Если данных нет, то записываем в пропущенный день
                if df.empty:
                    missed_days = read_json_file("missed_days.json")
                    missed_days.append({'date': formatted_date, 'market': market.value, 'endpoint': endpoint.value})
                    with open("missed_days.json", 'w') as f:
                        json.dump(missed_days, f, indent=4)
                    continue
                
                # считаем кол-во уникальных тикеров и дампим в json
                unique_tickers_count = df['secid'].nunique()
                tickers_count = read_json_file("tickers_count.json")
                tickers_count.append({'date': formatted_date, 'market': market.value, 'endpoint': endpoint.value, 'count': unique_tickers_count})
                with open("tickers_count.json", 'w') as f:
                    json.dump(tickers_count, f, indent=4)

                # считаем кол-во тикеров для 5минуток + график   
                count_5min_df = count_5min_tickers(df=df, market=market.value, endpoint=endpoint.value, date=formatted_date)
                count_5min_df.to_csv(f"report/5min_report/{formatted_date}_{market.value}_{endpoint.value}.csv", index=False, sep=";")
                plot_5min_tickers(df=count_5min_df, date=trading_date, market=market.value, endpoint=endpoint.value, filepath=f"report/5min_report/{formatted_date}_{market.value}_{endpoint.value}.png")

                # Валидируем рендж для imbalance и тд + неотрицательность для ценовых параметров и прочих не отрицательных значений
                negative_range_report = validate_range_negative_data(df=df, market=market.value, endpoint=endpoint.value)
                if negative_range_report is not False:
                    negative_range_report.to_csv(f"report/negative_range_report/{formatted_date}_{market.value}_{endpoint.value}.csv", index=False, sep=";")

                # Валидация округлений цен инструментов 
                # Нужно делать из внутренней сети, тк Decimals не отображается вне контура MOEX 
                
                # dataset = {
                #     formatted_date: {
                #         market.value: {
                #             endpoint.value: df
                #         }
                #     }
                # }
                # decimals_report = validate_decimals(dataset=dataset)
                # if not decimals_report.empty:
                #     decimals_report.to_csv(f"report/decimals_report/{formatted_date}_{market.value}_{endpoint.value}.csv", index=False, sep=";")b

In [ ]:
# Запуск отчета
START_DATE=date(2024, 6, 1)
END_DATE=date(2024, 6, 3)
get_dataset(start=START_DATE, end=END_DATE)

In [ ]:
# Формуируем отчет по пропущенным дням
missed_days = read_json_file("missed_days.json")
if len(missed_days) > 0:
    missed_days_df = pd.DataFrame(missed_days)
    missed_days_df = missed_days_df.groupby(['date', 'market'])['endpoint'].apply(list).reset_index()
    missed_days_df.to_csv("report/missed_days.csv", index=False, sep=";")
    missed_days_df
else:
    with open("report/missed_days.csv", 'w') as f:
        f.write("No missed days")

In [ ]:
# Формуируем отчет по количеству тикеров
with open("tickers_count.json", 'r') as file:
    data = json.load(file)
tickers_count_df = pd.DataFrame(data)
tickers_count_df.to_csv("report/tickers_count.csv", index=False, sep=";")
tickers_count_df

In [ ]:
plot_tickers_count_by_day(data=tickers_count_df, markets=['eq', 'fx', 'fo'], filepath="report/tickers_count_by_day.png")

In [ ]:
plot_tickers_count_by_day(data=tickers_count_df, markets=['fx'], filepath="report/tickers_count_by_day_fx.png")

In [ ]:
plot_tickers_count_by_day(data=tickers_count_df, markets=['fo'], filepath="report/tickers_count_by_day_fo.png")

In [ ]:
plot_tickers_count_by_day(data=tickers_count_df, markets=['eq'], filepath="report/tickers_count_by_day_eq.png")